In [1]:
import numpy as np
import datetime
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')

The Class

In [2]:
# A completely empty class
class StepByStep(object): 
    pass


The constructor : Define parts(attributes) that make up the class  

 Typical attributes include:
 * arguments provided by user
 * placehlders for other objects that are currently unavailable
 * variables that needed be tracked of
 * functions that are dynamically built

### 1. Arguments

In [3]:
class StepByStep(object):
    def __init__(self,model,loss_fn,optimizer):
        # Here we use define the attributes of our class
        # We start by storing the arguments as attributes to use later
        self.model=model
        self.loss_fn=loss_fn
        self.optimizer=optimizer
        self.device='cuda' if torch.cuda.is_aviailable() else 'cpu'
        # lets send the model to the device
        self.model.to(self.device)

    def to(self,device):
        # This method allows the user to specify a different device
        # It sets corresponding attribute and sends the model to device
        try:
            self.device=device
            self.model.to(self.device)
        except RuntimeError: # runtime error is raised if the device is not avilaible
            self.device=('cuda' if torch.cuda.is_available() else 'cpu')
            print(f"Couldnt send it to {device}, sending it to {self.device} instead")
            self.model.to(self.device)

### 2. Placeholders/delayed arguments
ex: Train and val loaders, summary writer

In [8]:
class StepByStep(object):
    # constructor
    def __init__(self,model,loss_fn,optimizer):
        # Here we use define the attributes of our class
        # We start by storing the arguments as attributes to use later
        self.model=model
        self.loss_fn=loss_fn
        self.optimizer=optimizer
        self.device='cuda' if torch.cuda.is_aviailable() else 'cpu'
        # Placeholders
        self.train_loader=None
        self.val_loader=None
        self.writer=None
        # lets send the model to the device
        self.model.to(self.device)

    def to(self,device):
        # This method allows the user to specify a different device
        # It sets corresponding attribute and sends the model to device
        try:
            self.device=device
            self.model.to(self.device)
        except RuntimeError: # runtime error is raised if the device is not avilaible
            self.device=('cuda' if torch.cuda.is_available() else 'cpu')
            print(f"Couldnt send it to {device}, sending it to {self.device} instead")
            self.model.to(self.device)

    def set_loaders(self,train_loader,val_loader=None):
        # This method allows the user to specify the which loader to use
        # Both loaders are then assign to corresponding attributes
        self.train_loader=train_loader
        self.val_loader=val_loader

    def set_tensorboard(self,name,folder='runs'):
        # This method allows the user to create a summarywriter to interface with tensorboard.
        suffix=datetime.datetime.now().strftime('%Y%m%d%H%M%S')
        self.writer=SummaryWriter(f"{folder}/{name}_{suffix}")
 



### 3. Variables  
### 4. Functions

In [12]:
class StepByStep(object):
    # constructor
    def __init__(self,model,loss_fn,optimizer):
        # Here we use define the attributes of our class
        # We start by storing the arguments as attributes to use later
        self.model=model
        self.loss_fn=loss_fn
        self.optimizer=optimizer
        self.device='cuda' if torch.cuda.is_aviailable() else 'cpu'
        # Placeholders
        self.train_loader=None
        self.val_loader=None
        self.writer=None
        # variables that needs to be keep track of
        self.losses=[]
        self.val_losses=[]
        self.total_epochs=0
        # create  train step function for model, loss function and optimizer
        self.train_step_fn=self._make_train_step_fn() # There are no arguments. It make use of class attributes directly
        # create validation step function for model and loss function
        self.val_step_fn=self._make_val_step_fn()
        # lets send the model to the device
        self.model.to(self.device)

    def to(self,device):
        # This method allows the user to specify a different device
        # It sets corresponding attribute and sends the model to device
        try:
            self.device=device
            self.model.to(self.device)
        except RuntimeError: # runtime error is raised if the device is not avilaible
            self.device=('cuda' if torch.cuda.is_available() else 'cpu')
            print(f"Couldnt send it to {device}, sending it to {self.device} instead")
            self.model.to(self.device)

    def set_loaders(self,train_loader,val_loader=None):
        # This method allows the user to specify the which loader to use
        # Both loaders are then assign to corresponding attributes
        self.train_loader=train_loader
        self.val_loader=val_loader

    def set_tensorboard(self,name,folder='runs'):
        # This method allows the user to create a summarywriter to interface with tensorboard.
        suffix=datetime.datetime.now().strftime('%Y%m%d%H%M%S')
        self.writer=SummaryWriter(f"{folder}/{name}_{suffix}")
 